###pre

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import Lasso,Ridge
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
import matplotlib.pyplot as plt



In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [ ]:
train_x = train_df.drop(columns = ['TIMESTAMP','PRODUCT_ID', 'Y_Quality','Y_Class'])
train_y = train_df[['Y_Quality','LINE']]
test_x = test_df.drop(columns = ['TIMESTAMP','PRODUCT_ID'])

In [ ]:
product_lst = ['A_31','T_31','O_31']
line_lst = ['T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305']

In [ ]:
train_df['LINE'].unique()

array(['T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305'],
      dtype=object)

In [ ]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    if i == 'LINE':
      print(le.transform(line_lst))
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

[2 3 4 5 1 0]
Done.


In [ ]:
train_a = train_x[train_x['LINE'] == 2]
train_b = train_x[train_x['LINE'] == 3]
train_c = train_x[train_x['LINE'] == 4]
train_d = train_x[train_x['LINE'] == 5]
train_e = train_x[train_x['LINE'] == 1]
train_f = train_x[train_x['LINE'] == 0]


train_a_y = train_y[train_y['LINE'] == line_lst[0]]
train_b_y = train_y[train_y['LINE'] == line_lst[1]]
train_c_y = train_y[train_y['LINE'] == line_lst[2]]
train_d_y = train_y[train_y['LINE'] == line_lst[3]]
train_e_y = train_y[train_y['LINE'] == line_lst[4]]
train_f_y = train_y[train_y['LINE'] == line_lst[5]]
train_a_y = train_a_y['Y_Quality']
train_b_y = train_b_y['Y_Quality']
train_c_y = train_c_y['Y_Quality']
train_d_y = train_d_y['Y_Quality']
train_e_y = train_e_y['Y_Quality']
train_f_y = train_f_y['Y_Quality']

test_a = test_x[test_x['LINE'] == 2]
test_b = test_x[test_x['LINE'] == 3]
test_c = test_x[test_x['LINE'] == 4]
test_d = test_x[test_x['LINE'] == 5]
test_e = test_x[test_x['LINE'] == 1]
test_f = test_x[test_x['LINE'] == 0]

col = train_x.columns
na_a_col = []
na_b_col = []
na_c_col = []
na_d_col = []
na_e_col = []
na_f_col = []
for i in col:
  if train_a[i].count() == 0:
    na_a_col.append(i)
  if train_b[i].count() == 0:
    na_b_col.append(i)
  if train_c[i].count() == 0:
    na_c_col.append(i)
  if train_d[i].count() == 0:
    na_d_col.append(i)
  if train_e[i].count() == 0:
    na_e_col.append(i)
  if train_f[i].count() == 0:
    na_f_col.append(i)

train_a = train_a.drop(na_a_col, axis = 1)
train_b = train_b.drop(na_b_col, axis = 1)
train_c = train_c.drop(na_c_col, axis = 1)
train_d = train_d.drop(na_d_col, axis = 1)
train_e = train_e.drop(na_e_col, axis = 1)
train_f = train_f.drop(na_f_col, axis = 1)

test_a = test_a.drop(na_a_col, axis = 1)
test_b = test_b.drop(na_b_col, axis = 1)
test_c = test_c.drop(na_c_col, axis = 1)
test_d = test_d.drop(na_d_col, axis = 1)
test_e = test_e.drop(na_e_col, axis = 1)
test_f = test_f.drop(na_f_col, axis = 1)
print(f"Shape of each Line : \n{train_a.shape}  \n{train_b.shape} \n{train_c.shape}\
 \n{train_d.shape} \n{train_e.shape} \n{train_f.shape}")

Shape of each Line : 
(78, 1970)  
(42, 1977) 
(175, 673) 
(174, 673) 
(70, 888) 
(59, 888)


In [ ]:
train_a = train_a.fillna(0)
train_b = train_b.fillna(0)
train_c = train_c.fillna(0)
train_d = train_d.fillna(0)
train_e = train_e.fillna(0)
train_f = train_f.fillna(0)

test_a = test_a.fillna(0)
test_b = test_b.fillna(0)
test_c = test_c.fillna(0)
test_d = test_d.fillna(0)
test_e = test_e.fillna(0)
test_f = test_f.fillna(0)

In [ ]:
# 각 라인별로 특성값이 하나 밖에 없는 값 제거
train_total = [train_a, train_b, train_c, train_d, train_e, train_f]
test_total = [test_a,test_b,test_c,test_d,test_e,test_f]
for idx, t_df in enumerate(train_total):
  tmp = []
  for i in t_df:
    if i == 'LINE' or i == 'PRODUCT_CODE':
      continue
    if len(t_df[i].unique()) < 2:
      tmp.append(i)
  t_df.drop(tmp, axis = 1, inplace = True)
  test_total[idx].drop(tmp, axis = 1, inplace = True)
  print(f" {t_df.shape}")

 (78, 1722)
 (42, 1726)
 (175, 571)
 (174, 543)
 (70, 733)
 (59, 732)


### 라쏘

In [ ]:
RF_a = RandomForestRegressor(max_depth = 10, min_samples_leaf = 2,\
          min_samples_split = 10,n_estimators = 10, n_jobs = -1).fit(train_a, train_a_y)
RF_b = RandomForestRegressor(max_depth = 40, min_samples_leaf = 4,\
          min_samples_split = 10,n_estimators = 20, n_jobs = -1).fit(train_b, train_b_y)
RF_c = RandomForestRegressor(max_depth = 20, min_samples_leaf = 10,\
          min_samples_split = 50,n_estimators = 20, n_jobs = -1).fit(train_c, train_c_y)
RF_d = RandomForestRegressor(max_depth = 20, min_samples_leaf = 2,\
          min_samples_split = 20,n_estimators = 20, n_jobs = -1).fit(train_d, train_d_y)
RF_e = RandomForestRegressor(max_depth = 30, min_samples_leaf = 4,\
          min_samples_split = 10,n_estimators = 10, n_jobs = -1).fit(train_e, train_e_y)
RF_f = RandomForestRegressor(max_depth = 12, min_samples_leaf = 2,\
          min_samples_split = 20,n_estimators = 30, n_jobs = -1).fit(train_f, train_f_y)

In [ ]:
# from sklearn.model_selection import GridSearchCV

# params = { 'n_estimators' : [10,100,500,1000,2000],
#            'max_depth' : [ 10, 12,14,16],
#            'min_samples_leaf' : [8,10,12,14],
#            'min_samples_split' : [8,12, 16, 20]
#             }
# grid_cv = GridSearchCV(RF_a, param_grid = params, cv = 3, n_jobs = -1)
# grid_cv.fit(train_a, train_a_y)
# print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
# print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

# grid_cv = GridSearchCV(RF_b, param_grid = params, cv = 3, n_jobs = -1)
# grid_cv.fit(train_b, train_b_y)
# print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
# print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

# grid_cv = GridSearchCV(RF_c, param_grid = params, cv = 3, n_jobs = -1)
# grid_cv.fit(train_c, train_c_y)
# print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
# print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

# grid_cv = GridSearchCV(RF_d, param_grid = params, cv = 3, n_jobs = -1)
# grid_cv.fit(train_d, train_d_y)
# print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
# print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

# grid_cv = GridSearchCV(RF_e, param_grid = params, cv = 3, n_jobs = -1)
# grid_cv.fit(train_e, train_e_y)
# print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
# print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

# grid_cv = GridSearchCV(RF_f, param_grid = params, cv = 3, n_jobs = -1)
# grid_cv.fit(train_f, train_f_y)
# print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
# print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

In [ ]:
pred_a = RF_a.predict(test_a)
pred_b = RF_b.predict(test_b)
pred_c = RF_c.predict(test_c)
pred_d = RF_d.predict(test_d)
pred_e = RF_e.predict(test_e)
pred_f = RF_f.predict(test_f)

###pre

In [ ]:
train_x = train_df.drop(columns = ['TIMESTAMP','PRODUCT_ID','Y_Class'])
train_y = train_df[['Y_Class','LINE']]
test_x = test_df.drop(columns = ['TIMESTAMP','PRODUCT_ID'])

In [ ]:
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    if i == 'LINE':
      print(le.transform(line_lst))
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

[2 3 4 5 1 0]
Done.


In [ ]:
train_a = train_x[train_x['LINE'] == 2]
train_b = train_x[train_x['LINE'] == 3]
train_c = train_x[train_x['LINE'] == 4]
train_d = train_x[train_x['LINE'] == 5]
train_e = train_x[train_x['LINE'] == 1]
train_f = train_x[train_x['LINE'] == 0]


train_a_y = train_y[train_y['LINE'] == line_lst[0]]
train_b_y = train_y[train_y['LINE'] == line_lst[1]]
train_c_y = train_y[train_y['LINE'] == line_lst[2]]
train_d_y = train_y[train_y['LINE'] == line_lst[3]]
train_e_y = train_y[train_y['LINE'] == line_lst[4]]
train_f_y = train_y[train_y['LINE'] == line_lst[5]]
train_a_y = train_a_y['Y_Class']
train_b_y = train_b_y['Y_Class']
train_c_y = train_c_y['Y_Class']
train_d_y = train_d_y['Y_Class']
train_e_y = train_e_y['Y_Class']
train_f_y = train_f_y['Y_Class']

test_a = test_x[test_x['LINE'] == 2]
test_b = test_x[test_x['LINE'] == 3]
test_c = test_x[test_x['LINE'] == 4]
test_d = test_x[test_x['LINE'] == 5]
test_e = test_x[test_x['LINE'] == 1]
test_f = test_x[test_x['LINE'] == 0]

# col = train_x.columns
# na_a_col = []
# na_b_col = []
# na_c_col = []
# na_d_col = []
# na_e_col = []
# na_f_col = []
# for i in col:
#   if train_a[i].count() == 0:
#     na_a_col.append(i)
#   if train_b[i].count() == 0:
#     na_b_col.append(i)
#   if train_c[i].count() == 0:
#     na_c_col.append(i)
#   if train_d[i].count() == 0:
#     na_d_col.append(i)
#   if train_e[i].count() == 0:
#     na_e_col.append(i)
#   if train_f[i].count() == 0:
#     na_f_col.append(i)

# train_a = train_a.drop(na_a_col, axis = 1)
# train_b = train_b.drop(na_b_col, axis = 1)
# train_c = train_c.drop(na_c_col, axis = 1)
# train_d = train_d.drop(na_d_col, axis = 1)
# train_e = train_e.drop(na_e_col, axis = 1)
# train_f = train_f.drop(na_f_col, axis = 1)

# test_a = test_a.drop(na_a_col, axis = 1)
# test_b = test_b.drop(na_b_col, axis = 1)
# test_c = test_c.drop(na_c_col, axis = 1)
# test_d = test_d.drop(na_d_col, axis = 1)
# test_e = test_e.drop(na_e_col, axis = 1)
# test_f = test_f.drop(na_f_col, axis = 1)

In [ ]:
train_a = train_a.fillna(0)
train_b = train_b.fillna(0)
train_c = train_c.fillna(0)
train_d = train_d.fillna(0)
train_e = train_e.fillna(0)
train_f = train_f.fillna(0)

test_a = test_a.fillna(0)
test_b = test_b.fillna(0)
test_c = test_c.fillna(0)
test_d = test_d.fillna(0)
test_e = test_e.fillna(0)
test_f = test_f.fillna(0)

In [ ]:
# 각 라인별로 특성값이 하나 밖에 없는 값 제거
train_total = [train_a, train_b, train_c, train_d, train_e, train_f]
test_total = [test_a, test_b,test_c,test_d,test_e,test_f]
for idx, t_df in enumerate(train_total):
  tmp = []
  for i in t_df:
    if i == 'LINE' or i == 'PRODUCT_CODE':
      continue
    if len(t_df[i].unique()) < 2:
      tmp.append(i)
  t_df.drop(tmp, axis = 1, inplace = True)
  test_total[idx].drop(tmp, axis = 1, inplace = True)

In [ ]:
test_a['Y_Quality'] = pred_a
test_b['Y_Quality'] = pred_b
test_c['Y_Quality'] = pred_c
test_d['Y_Quality'] = pred_d
test_e['Y_Quality'] = pred_e
test_f['Y_Quality'] = pred_f

In [ ]:
new_cola = ['Y_Quality'] + test_a.columns.to_list()[:-1]
new_colb = ['Y_Quality'] + test_b.columns.to_list()[:-1]
new_colc = ['Y_Quality'] + test_c.columns.to_list()[:-1]
new_cold = ['Y_Quality'] + test_d.columns.to_list()[:-1]
new_cole = ['Y_Quality'] + test_e.columns.to_list()[:-1]
new_colf = ['Y_Quality'] + test_f.columns.to_list()[:-1]
test_a = test_a[new_cola]
test_b = test_b[new_colb]
test_c = test_c[new_colc]
test_d = test_d[new_cold]
test_e = test_e[new_cole]
test_f = test_f[new_colf]

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# train_total = [train_a, train_b, train_c, train_d, train_e, train_f]
# test_total = [test_a, test_b,test_c,test_d,test_e,test_f]
# for k in range(len(train_total)):
#   print(test_total[k].shape)
#   for i in train_total[k].columns:
#     mm = MinMaxScaler()
#     fitted = mm.fit(train_total[k][i].values.reshape(-1,1))
#     output = mm.transform(train_total[k][i].values.reshape(-1,1))
#     out_test = mm.transform(test_total[k][i].values.reshape(-1,1))
#     test_total[k][i] = out_test.reshape(-1)
#     train_total[k][i] = output.reshape(-1)
#   print(test_total[k].shape)

#    train

In [ ]:
!pip install pytorch-tabnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 3.8 MB/s eta 0:00:00


In [ ]:
# 트리 기반
from sklearn.model_selection import KFold
import optuna
from optuna import Trial, visualization

def Objective(trial):
    mask_type = trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    n_da = trial.suggest_int("n_da", 56, 64, step=4)
    n_steps = trial.suggest_int("n_steps", 1, 3, step=1)
    gamma = trial.suggest_float("gamma", 1., 1.4, step=0.2)
    n_shared = trial.suggest_int("n_shared", 1, 3)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
    
    tabnet_params = dict(n_d=n_da, n_a=n_da, n_steps=n_steps, gamma=gamma,
                     lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     mask_type=mask_type, n_shared=n_shared,
                     scheduler_params=dict(mode="min",
                                           patience=trial.suggest_int("patienceScheduler",low=3,high=10), # changing sheduler patience to be lower than early stopping patience 
                                           min_lr=1e-5,
                                           factor=0.5,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=True
                     ) #early stopping
    #valid
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    CV_score_array    =[]

    for train_index, test_index in kf.split(train_a):
        X_train, X_valid = train_a[train_index], train_a[test_index]
        y_train, y_valid = train_a_y[train_index], train_a_y[test_index]
        clf = TabNetClassifier(**tabnet_params)
        clf.fit(X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  patience=trial.suggest_int("patience",low=15,high=30), max_epochs=trial.suggest_int('epochs', 1, 100),
                  eval_metric=['accuracy'])
        CV_score_array.append(clf.best_cost)
    avg = np.mean(CV_score_array)
    return avg


study = optuna.create_study(direction="minimize", study_name='TabNet optimization')
study.optimize(Objective, timeout=6*60) #5 hours

TabNet_params = study.best_params
print(TabNet_params)


KeyboardInterrupt: ignored

In [ ]:
for i in range(6):
  print(y_preds[i])

[0 0 0 0 2 1 0 0 0 2 2 0 2]
[0 1 0 0 2 2 2 2 2 2 2 2 2 2 1 2 2 0 2 1 2 2 2 2 1 1]
[1 1 1 2 2 1 1 1 1 2 2 2 2 2 2 2 2 1 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 0 2 2 2 2 2 1 1 1 2 2 1 1 1 1 1 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 2 2 2 2 2 2 2 2 2 2]
[1 1 1 2 0 1 1 1 2 2 0 2 1 1 1 1 1 2 2 2 2 2 0 1 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 2 2 0 2 2 0 2 1 2 2 2 2 2 2 2 2 2 0 1 2 0 2 0 0 0 0 2 0
 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 2]
[1 1 1 1 0 2 2 1 2 0 2 2 0 0]
[0 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [ ]:
for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

Done.


In [ ]:
for i in range(6):
  y_preds[i] = y_preds[i]

AttributeError: ignored

In [ ]:
test_pred = []
cnt = [0 for i in range(6)]
test_prd = test_x['LINE']
for idx,i in enumerate(test_prd):
  if i == 2:
    test_pred.append(y_preds[0][cnt[0]])
    cnt[0] += 1
  elif i == 3:
    test_pred.append(y_preds[1][cnt[1]])
    cnt[1] += 1
  elif i == 4:
    test_pred.append(y_preds[2][cnt[2]])
    cnt[2] += 1
  elif i == 5:
    test_pred.append(y_preds[3][cnt[3]])
    cnt[3] += 1
  elif i == 1:
    test_pred.append(y_preds[4][cnt[4]])
    cnt[4] += 1
  else:
    test_pred.append(y_preds[5][cnt[5]])
    cnt[5] += 1

In [ ]:
for i in range(len(test_pred)):
  print(test_pred[i])

0
1
1
0
0
0
0
2
2
0
0
0
0
0
0
1
0
0
2
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
2
2
0
0
0
0
0
0
0
0
0
2
2
2
1
0
0
1
1
0
2
0
0
2
1
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
2
0
0
2
0
2
0
2
1
2
1
2
0
2
2
0
2
0
2
2
2
2
1
0
0
0
0
2
0
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
1
0
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


#save

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = test_pred
path = 'baselinne.csv'
submit.to_csv(path, index = False)

ImportError: ignored

In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
